In [2]:
import sys
!{sys.executable} -m pip install minio


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\aayushs2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!{sys.executable} -m pip install flask


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\aayushs2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
!{sys.executable} -m pip install python-dotenv


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: C:\Users\aayushs2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import os
from datetime import datetime
from minio import Minio
from dotenv import load_dotenv

In [6]:
load_dotenv()
ACCESS_KEY = os.environ.get('ACCESS_KEY')
SECRET_KEY = os.environ.get('SECRET_KEY')
BUCKET_NAME = os.environ.get('BUCKET_NAME')
minIO_host = "ec2-65-0-177-178.ap-south-1.compute.amazonaws.com:9000"
minIO_client = Minio(minIO_host, access_key=ACCESS_KEY, secret_key=SECRET_KEY, secure=False)

In [7]:
# checking and making buckets if it doesn't exits
for i in ["source", "processed"]:
    if minIO_client.bucket_exists(i):
        print(f"{i} exists")
    else:
        print(f"making bucket {i}")
        minIO_client.make_bucket(i)

source exists
processed exists


In [8]:
# listing all the buckets
for bucket in minIO_client.list_buckets():
    print(bucket.name)

data
processed
source


In [9]:
# list all objects in a bucket
for i in minIO_client.list_objects('data', recursive=True):
    print(i.object_name)

processed/data-marts/README.md
processed/feature-stores/README.md
processed/raw/README.md
processed/summaries/README.md
source/tpcds/README.md
source/tpcds/store_sales_1669852035.377949.csv.gz
source/tpcds/store_sales_1669853423.776755.csv.gz
source/tpcds/store_sales_1669912850.295695.csv.gz
source/tpcds/store_sales_1669978104.35895.csv.gz
source/tpcds/store_sales_1669978544.322109.csv.gz
source/tpcds/store_sales_1669979211.421977.csv.gz
source/tpcds/store_sales_1669993596.592191.csv.gz
source/tpcds/store_sales_1669993917.438838.csv.gz
source/tpcds/store_sales_1669994452.711803.csv.gz
source/tpcds/store_sales_1669994517.341453.csv.gz


In [10]:
# upload data with metadata
result = minIO_client.fput_object(
    "source", "tpcds/store_sales_1669653618.908919.csv.gz", "/home/aayush/bpcl/store_sales_1669653618.908919.csv.gz",
    metadata={"table" : "store_sales", "datetime" : datetime.now(), "source" : "tpcds", "job_type" : "python_store_sales_extract"}
)
print(f"{result.object_name} {result.etag} {result.version_id}")

tpcds/store_sales_1669653618.908919.csv.gz 90b68718e038258d5b6c2664c35a3ad3 None


In [11]:
# tagging objects
from minio.commonconfig import Tags
tags = Tags.new_bucket_tags()
tags["table"] = "store_sales"
tags["datetime"] = str(datetime.now())
tags["source"] = "tpcds"
minIO_client.set_object_tags("source", "tpcds/store_sales_1669653618.908919.csv.gz", tags)

In [50]:
# listing object
object = minIO_client.list_objects("source", recursive=True)
for obj in object:
    print(type(obj.object_name))
    print(obj.metadata)
    print(list(obj.metadata.keys()))

<class 'str'>
{}
[]


In [51]:
# listing with metadata
object = minIO_client.list_objects("source", include_user_meta=True, recursive=True)
for obj in object:
    print(type(obj.object_name))
    print(obj.metadata)
    print(list(obj.metadata.keys()))

<class 'str'>
{'X-Amz-Meta-Table': 'store_sales', 'X-Amz-Meta-Job_type': 'python_store_sales_extract', 'X-Amz-Meta-Datetime': '2022-12-02 21:29:57.729817', 'X-Amz-Meta-Source': 'tpcds', 'content-type': 'application/octet-stream'}
['X-Amz-Meta-Table', 'X-Amz-Meta-Job_type', 'X-Amz-Meta-Datetime', 'X-Amz-Meta-Source', 'content-type']


In [53]:
# getting tags
objects = minIO_client.list_objects("source", recursive=True)
for i in objects:
    tagss = minIO_client.get_object_tags("source", i.object_name)
    print(tagss)

{'source': 'tpcds', 'table': 'store_sales', 'datetime': '2022-12-02 21:29:58.090984'}


In [54]:
# filtering using tags
objects = minIO_client.list_objects("source", recursive=True)
for i in objects:
    tagss = minIO_client.get_object_tags("source", i.object_name)
    if 'tpcds' in list(tagss.values()):
        print(obj.object_name)

tpcds/store_sales_1669653618.908919.csv.gz
